In [2]:
# import packages
import torch
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd
import numpy as np

In [4]:
dataset = pd.read_csv("./data/storepurchasedata_large.csv")

In [5]:
dataset.shape

(1554, 3)

In [6]:
dataset.head()

,Age,Salary,Purchased
0,18,20000,0
1,19,22000,0
2,20,24000,0
3,21,28000,0
4,22,60000,1


In [7]:
# Get the IDV and DV in X and y variables
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:,-1].values

In [8]:
# Split the data to train & test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20,random_state=0)

In [9]:
# Standard scaling the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
# convert the arrays to the tensors IDV's
Xtrain_ = torch.from_numpy(X_train).float()
Xtest_ = torch.from_numpy(X_test).float()

In [13]:
# DV to Tensor
ytrain_ = torch.from_numpy(y_train)
ytest_ = torch.from_numpy(y_test)

In [15]:
Xtrain_.shape, ytrain_.shape, Xtest_.shape, ytest_.shape

(torch.Size([1243, 2]),
 torch.Size([1243]),
 torch.Size([311, 2]),
 torch.Size([311]))

<img src="./data/nn-image.png">

In [16]:
# Fix the neural network parameters
input_size=2
output_size=2
hidden_size=10

In [18]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
        self.fc3 = torch.nn.Linear(hidden_size, output_size)


    def forward(self, X):
        X = torch.relu((self.fc1(X)))
        X = torch.relu((self.fc2(X)))
        X = self.fc3(X)

        return F.log_softmax(X,dim=1)

In [19]:
model = Net()
import torch.optim as optim
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()

In [20]:
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    Ypred = model(Xtrain_)
    loss = loss_fn(Ypred,  ytrain_)
    loss.backward()
    optimizer.step()
    print('Epoch',epoch, 'loss',loss.item())

list(model.parameters())

Epoch 0 loss 0.7116402983665466
Epoch 1 loss 0.6918826699256897
Epoch 2 loss 0.6724310517311096
Epoch 3 loss 0.6534146666526794
Epoch 4 loss 0.6349195241928101
Epoch 5 loss 0.616766631603241
Epoch 6 loss 0.5985733866691589
Epoch 7 loss 0.5800601840019226
Epoch 8 loss 0.5609275102615356
Epoch 9 loss 0.5412424206733704
Epoch 10 loss 0.5212754607200623
Epoch 11 loss 0.5009044408798218
Epoch 12 loss 0.479924738407135
Epoch 13 loss 0.4585697054862976
Epoch 14 loss 0.4369315207004547
Epoch 15 loss 0.4152473211288452
Epoch 16 loss 0.3933025300502777
Epoch 17 loss 0.3716881573200226
Epoch 18 loss 0.35023316740989685
Epoch 19 loss 0.32891732454299927
Epoch 20 loss 0.308023601770401
Epoch 21 loss 0.28803542256355286
Epoch 22 loss 0.2696225941181183
Epoch 23 loss 0.2530255913734436
Epoch 24 loss 0.23744407296180725
Epoch 25 loss 0.22291919589042664
Epoch 26 loss 0.20957732200622559
Epoch 27 loss 0.1974719762802124
Epoch 28 loss 0.1864442080259323
Epoch 29 loss 0.17649470269680023
Epoch 30 loss 0.

[Parameter containing:
 tensor([[ 1.1845,  0.1708],
         [ 1.0688,  0.0491],
         [-0.1095,  0.2725],
         [-0.4175, -0.7834],
         [-0.7833,  0.7674],
         [-0.8426, -0.7962],
         [-0.1553,  0.6721],
         [-0.2452,  0.6762],
         [ 0.6899, -0.0326],
         [-0.4535, -0.6424]], requires_grad=True),
 Parameter containing:
 tensor([ 0.4757,  0.6939, -0.3026,  0.6071,  0.3092, -0.0562,  0.4169,  0.3273,
          0.0891,  0.4045], requires_grad=True),
 Parameter containing:
 tensor([[ 0.2839,  0.6936,  0.4528, -0.4053,  0.0238, -0.5775,  0.1022,  0.2906,
           0.5569, -0.2428],
         [-0.0976, -0.1882,  0.0789, -0.3716,  0.0352, -0.2724,  0.1556, -0.2126,
           0.2017, -0.2253],
         [-0.3919, -0.0543, -0.0109,  0.3189, -0.3510,  0.2400,  0.0653, -0.3234,
          -0.3637,  0.4468],
         [ 0.1579, -0.2660,  0.0229, -0.0661, -0.2295,  0.1056, -0.3102, -0.1969,
           0.2503, -0.3125],
         [ 0.1455, -0.1903, -0.0930, -0.0610,

## Predict using Pytorch Models

In [21]:
torch.from_numpy(sc.transform(np.array([[40,20000]]))).float()

tensor([[-0.2650, -1.7622]])

In [22]:
y_cust_20_40000 = model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-4.1995e-03, -5.4749e+00]], grad_fn=<LogSoftmaxBackward0>)

In [24]:
_, predicted_20_40000 = torch.max(y_cust_20_40000.data,-1)
predicted_20_40000

tensor([0])

In [25]:
y_cust_42_50000 = model(torch.from_numpy(sc.transform(np.array([[42,50000]]))).float())
y_cust_42_50000

tensor([[-1.0337, -0.4396]], grad_fn=<LogSoftmaxBackward0>)

In [26]:
_, predicted_42_50000 = torch.max(y_cust_42_50000.data,-1)
predicted_42_50000

tensor([1])

## Save the model file

In [30]:
torch.save(model, 'customer_buy.pt')

In [31]:
!ls

'Building_&_Deploy_Pytorch.ipynb'   customer_buy_state_dict
 customer_buy.pt		    data


In [32]:
# Load the model
restored_model = torch.load('customer_buy.pt')

In [33]:
# Predict from loaded model
y_cust_42_50000 = restored_model(torch.from_numpy(sc.transform(np.array([[42,50000]]))).float())
y_cust_42_50000

tensor([[-1.0337, -0.4396]], grad_fn=<LogSoftmaxBackward0>)

In [34]:
_, predicted_42_50000 = torch.max(y_cust_42_50000.data,-1)
predicted_42_50000

tensor([1])

## Recommended way of saving the model 

In [27]:
# Save the torch model file
torch.save(model.state_dict(),'customer_buy_state_dict')

## Use pytorch dictionary

In [35]:
input_size=2
output_size=2
hidden_size=10

In [36]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
        self.fc3 = torch.nn.Linear(hidden_size, output_size)


    def forward(self, X):
        X = torch.relu((self.fc1(X)))
        X = torch.relu((self.fc2(X)))
        X = self.fc3(X)

        return F.log_softmax(X,dim=1)

In [37]:
new_predictor = Net()

In [38]:
new_predictor.load_state_dict(torch.load('customer_buy_state_dict'))

<All keys matched successfully>

In [39]:
# Predict from loaded model
y_cust_42_50000 = new_predictor(torch.from_numpy(sc.transform(np.array([[42,50000]]))).float())
y_cust_42_50000

tensor([[-1.0337, -0.4396]], grad_fn=<LogSoftmaxBackward0>)

In [40]:
_, predicted_42_50000 = torch.max(y_cust_42_50000.data,-1)
predicted_42_50000

tensor([1])